# pyopensky demo

## Setup

In [58]:
!python -m pip install -q pyopensky folium awswrangler

## Imports

In [50]:
from pyopensky.config import opensky_config_dir
from pyopensky.trino import Trino
import pandas as pd
import awswrangler as wr
from glob import glob
import os
import datetime
import folium
from folium import Popup
from tqdm import tqdm
from uas_risk_analysis_packages.eda import get_missing_values
from uas_risk_analysis_packages.ml import functions as F

In [51]:
os.environ['AWS_DEFAULT_PROFILE'] = 'endurasoft-dev'

In [52]:
# Verify connection to s3
!aws s3 ls s3://endurasoft-dev-risk-framework/opensky-network/sample/

2024-10-04 08:08:55 21914099421 20170109_16_anonymized.avro


## Configurations

In [53]:
# Must add credentials to pyopensky settings.conf file
glob(os.path.join(opensky_config_dir, '*.conf'))

['/Users/brettallen/Library/Application Support/pyopensky/settings.conf']

In [54]:
trino = Trino()

## Get Airports

In [55]:
airports = pd.read_csv('../data_curation/datasets/airports/airports.csv')
airports.head()

,icao,iata,name,city,state,country,elevation,lat,lon,tz
0,00AK,NaN,Lowell Field,Anchor Point,Alaska,US,450,59.949200,-151.695999,America/Anchorage
1,00AL,NaN,Epps Airpark,Harvest,Alabama,US,820,34.864799,-86.770302,America/Chicago
2,00AZ,NaN,Cordes Airport,Cordes,Arizona,US,3810,34.305599,-112.165001,America/Phoenix
3,00CA,NaN,Goldstone /Gts/ Airport,Barstow,California,US,3038,35.350498,-116.888000,America/Los_Angeles
4,00CL,NaN,Williams Ag Airport,Biggs,California,US,87,39.427189,-121.763428,America/Los_Angeles


In [56]:
airports['country'].value_counts()

country
US    13574
BR     2826
AU     1609
CA     1170
DE      481
      ...  
AW        1
CW        1
SX        1
BM        1
IM        1
Name: count, Length: 235, dtype: int64

In [57]:
airports['country'].unique().tolist()

['US',
 'PR',
 'AQ',
 'SB',
 'NR',
 'PG',
 'GL',
 'IS',
 'KS',
 'CA',
 'DZ',
 'AU',
 'BJ',
 'BF',
 'GH',
 'CI',
 'NG',
 'TN',
 'NE',
 'TG',
 'BE',
 'DE',
 'EE',
 'FI',
 'GB',
 'GG',
 'JE',
 'IM',
 'FK',
 'NL',
 'IE',
 'DK',
 'FO',
 'LU',
 'NO',
 'PL',
 'SE',
 'LV',
 'LT',
 'ZA',
 'BW',
 'CG',
 'SZ',
 'CF',
 'GQ',
 'SH',
 'MU',
 'IO',
 'CM',
 'ZM',
 'MG',
 'KM',
 'YT',
 'RE',
 'AO',
 'GA',
 'ST',
 'MZ',
 'SC',
 'TD',
 'ZW',
 'MW',
 'LS',
 nan,
 'CD',
 'ML',
 'GM',
 'ES',
 'SL',
 'GW',
 'LR',
 'MA',
 'EH',
 'SN',
 'MR',
 'GN',
 'CV',
 'ET',
 'BI',
 'SO',
 'DJ',
 'EG',
 'ER',
 'KE',
 'LY',
 'RW',
 'SS',
 'SD',
 'TZ',
 'UG',
 'IQ',
 'AL',
 'BG',
 'CY',
 'HR',
 'FR',
 'PM',
 'GR',
 'HU',
 'IT',
 'SI',
 'CZ',
 'IL',
 'MT',
 'AT',
 'PT',
 'BA',
 'RO',
 'CH',
 'TR',
 'MD',
 'MK',
 'GI',
 'RS',
 'ME',
 'SK',
 'TC',
 'DO',
 'CR',
 'GT',
 'HN',
 'JM',
 'MH',
 'MX',
 'NI',
 'PA',
 'SV',
 'HT',
 'CU',
 'KY',
 'BS',
 'BZ',
 'CK',
 'FJ',
 'TO',
 'KI',
 'TV',
 'NU',
 'WF',
 'AS',
 'WS',
 'PF',
 'VU',


In [7]:
get_missing_values(airports)

,total_missing,total_available,ratio_missing,percent_missing
icao,0,29266,0.000000,0.0%
iata,21350,7916,0.729515,73.0%
name,0,29266,0.000000,0.0%
city,2889,26377,0.098715,9.9%
state,760,28506,0.025969,2.6%
country,63,29203,0.002153,0.2%
elevation,0,29266,0.000000,0.0%
lat,0,29266,0.000000,0.0%
lon,0,29266,0.000000,0.0%
tz,0,29266,0.000000,0.0%


In [8]:
airports[airports['icao'] == 'KBWI']

,icao,iata,name,city,state,country,elevation,lat,lon,tz
12605,KBWI,BWI,Baltimore/Washington International Thurgood Ma...,Baltimore,Maryland,US,146,39.1754,-76.668297,America/New_York


## Obtain Data From Trino Database
See https://open-aviation.github.io/pyopensky/trino.html
```python
# full description of the whole set of parameters in the documentation
trino.flightlist(start, stop, *, airport, callsign, icao24)
trino.history(start, stop, *, callsign, icao24, bounds)
trino.rawdata(start, stop, *, callsign, icao24, bounds)
```

In [9]:
start_datetime = datetime.datetime.strptime('2024-08-08 13:55:00', '%Y-%m-%d %H:%M:%S') # 2024-08-08 09:55:00 (EST), 2024-08-08 13:55:00 (UTC)
end_datetime = datetime.datetime.strptime('2024-08-20 20:35:00', '%Y-%m-%d %H:%M:%S')   # 2024-08-20 16:35:00 (EST), 2024-08-20 20:35:00 (UTC)

### Flight List (Flight Profile)

In [10]:
start = start_datetime
stop = start_datetime + datetime.timedelta(hours=1)

flightlist_results = trino.flightlist(
    start=start,
    stop=stop,
    # airport='KBWI',
)

print(f'Total flights between {start} and {stop}: {len(flightlist_results):,}')

Total flights between 2024-08-08 13:55:00 and 2024-08-08 14:55:00: 7,185


In [58]:
airports_set = set(airports[airports['country'] == 'US']['icao'].unique().tolist())
print(f'Unique airports: {len(airports_set)}')

Unique airports: 13574


In [12]:
# Randomly sample 100 flights where departure and arrival are known and belong in the airports list
flightlist_results[(flightlist_results['departure'].isin(airports_set)) & (flightlist_results['arrival'].isin(airports_set))].sample(100).head()

,icao24,firstseen,departure,lastseen,arrival,callsign,day
5403,3e7932,2024-08-08 12:32:41+00:00,EDAY,2024-08-08 13:58:39+00:00,EDVN,DKLCC,2024-08-08 00:00:00+00:00
6047,ab27d3,2024-08-08 13:01:59+00:00,KOMA,2024-08-08 14:45:44+00:00,KCCO,N818HR,2024-08-08 00:00:00+00:00
3448,8963f7,2024-08-08 06:43:58+00:00,OMDB,2024-08-08 14:15:29+00:00,VHHH,UAE380,2024-08-08 00:00:00+00:00
5307,aa68c4,2024-08-08 14:38:10+00:00,NE40,2024-08-08 14:49:13+00:00,KFBY,N77XJ,2024-08-08 00:00:00+00:00
6187,aa5c1c,2024-08-08 13:52:40+00:00,KONT,2024-08-08 14:38:09+00:00,KSBA,CFS8693,2024-08-08 00:00:00+00:00


In [13]:
flightlist_results[(flightlist_results['departure'] == 'KBWI') & (flightlist_results['arrival'] == 'KHOU')]

,icao24,firstseen,departure,lastseen,arrival,callsign,day
4360,aa73a0,2024-08-08 11:40:00+00:00,KBWI,2024-08-08 14:18:41+00:00,KHOU,SWA16,2024-08-08 00:00:00+00:00


### Flight History (Track Points)
See https://open-aviation.github.io/pyopensky/trino.html#pyopensky.trino.Trino.history

In [14]:
start = start_datetime
flight_id = 'aa73a0'

# NOTE: By default, date_delta splits requests by hour
history_results = trino.history(
    start=start,
    icao24=flight_id
)

print(f'History data records from {start} for flight id "{flight_id}": {len(history_results):,}')

History data records from 2024-08-08 13:55:00 for flight id "aa73a0": 31,826


In [15]:
history_results.head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,serials,hour
0,2024-08-08 16:00:02+00:00,aa73a0,32.306741,-96.210919,174.981304,314.404431,-10.07872,SWA16,False,False,False,<NA>,3505.2,3726.18,1723132801.654,1723132801.731,"[-1408236845, -1408237160]",2024-08-08 16:00:00+00:00
1,2024-08-08 16:00:03+00:00,aa73a0,32.307812,-96.212208,174.981304,314.404431,-10.07872,SWA16,False,False,False,<NA>,3497.58,3718.56,1723132802.671,1723132802.671,"[-1408236845, -1408237160]",2024-08-08 16:00:00+00:00
2,2024-08-08 16:00:04+00:00,aa73a0,32.308273,-96.212769,174.981304,314.404431,-10.07872,SWA16,False,False,False,<NA>,3489.96,3703.32,1723132803.126,1723132803.354,"[-1408236845, -1408237160]",2024-08-08 16:00:00+00:00
3,2024-08-08 16:00:05+00:00,aa73a0,32.310093,-96.214955,174.981304,314.404431,-10.07872,SWA16,False,False,False,<NA>,3474.72,3680.46,1723132804.684,1723132804.858,"[-1408236845, -1408237160]",2024-08-08 16:00:00+00:00
4,2024-08-08 16:00:06+00:00,aa73a0,32.311163,-96.216244,174.981304,314.404431,-10.07872,SWA16,False,False,False,<NA>,3467.1,3672.84,1723132805.694,1723132805.974,"[-1408236845, -1408237160]",2024-08-08 16:00:00+00:00


Based on flight track points data, records are 1 track point per second (1HZ)

In [16]:
history_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31826 entries, 0 to 31825
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype                         
---  ------         --------------  -----                         
 0   time           31826 non-null  timestamp[ns, tz=UTC][pyarrow]
 1   icao24         31826 non-null  string                        
 2   lat            31718 non-null  double[pyarrow]               
 3   lon            31718 non-null  double[pyarrow]               
 4   velocity       29041 non-null  double[pyarrow]               
 5   heading        29041 non-null  double[pyarrow]               
 6   vertrate       29041 non-null  double[pyarrow]               
 7   callsign       31741 non-null  string                        
 8   onground       31826 non-null  bool[pyarrow]                 
 9   alert          31826 non-null  bool[pyarrow]                 
 10  spi            31826 non-null  bool[pyarrow]                 
 11  squawk         

In [17]:
# Create partition columns for year, month, day, hour
history_results['year'] = history_results['time'].dt.year
history_results['month'] = history_results['time'].dt.month
history_results['day'] = history_results['time'].dt.day
history_results['hour'] = history_results['time'].dt.hour
history_results[['time', 'year', 'month', 'day', 'hour']].head()

,time,year,month,day,hour
0,2024-08-08 16:00:02+00:00,2024,8,8,16
1,2024-08-08 16:00:03+00:00,2024,8,8,16
2,2024-08-08 16:00:04+00:00,2024,8,8,16
3,2024-08-08 16:00:05+00:00,2024,8,8,16
4,2024-08-08 16:00:06+00:00,2024,8,8,16


In [18]:
history_results_missing_values = get_missing_values(history_results)
history_results_missing_values

,total_missing,total_available,ratio_missing,percent_missing
time,0,31826,0.000000,0.0%
icao24,0,31826,0.000000,0.0%
lat,108,31718,0.003393,0.3%
lon,108,31718,0.003393,0.3%
velocity,2785,29041,0.087507,8.8%
heading,2785,29041,0.087507,8.8%
vertrate,2785,29041,0.087507,8.8%
callsign,85,31741,0.002671,0.3%
onground,0,31826,0.000000,0.0%
alert,0,31826,0.000000,0.0%


In [19]:
history_results_missing_values.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, time to day
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   total_missing    21 non-null     int64  
 1   total_available  21 non-null     int64  
 2   ratio_missing    21 non-null     float64
 3   percent_missing  21 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.4+ KB


In [20]:
history_results_missing_values['ratio_missing'] = history_results_missing_values['percent_missing'].apply(lambda x: float(x.replace('%', ''))/100)
history_results_missing_values

,total_missing,total_available,ratio_missing,percent_missing
time,0,31826,0.000,0.0%
icao24,0,31826,0.000,0.0%
lat,108,31718,0.003,0.3%
lon,108,31718,0.003,0.3%
velocity,2785,29041,0.088,8.8%
heading,2785,29041,0.088,8.8%
vertrate,2785,29041,0.088,8.8%
callsign,85,31741,0.003,0.3%
onground,0,31826,0.000,0.0%
alert,0,31826,0.000,0.0%


In [21]:
# Forward fill null records for all columns that have fewer than 15% missing values
missing_data_cols = history_results_missing_values[history_results_missing_values['ratio_missing'] < 0.15].index.tolist()
for col in missing_data_cols:
    history_results[col] = history_results[col].ffill()

In [22]:
get_missing_values(history_results)

,total_missing,total_available,ratio_missing,percent_missing
time,0,31826,0.000000,0.0%
icao24,0,31826,0.000000,0.0%
lat,0,31826,0.000000,0.0%
lon,0,31826,0.000000,0.0%
velocity,0,31826,0.000000,0.0%
heading,0,31826,0.000000,0.0%
vertrate,0,31826,0.000000,0.0%
callsign,0,31826,0.000000,0.0%
onground,0,31826,0.000000,0.0%
alert,0,31826,0.000000,0.0%


In [23]:
# Identify gaps in time
sorted_history = history_results.sort_values('time', ascending=True)
sorted_history['time_diff'] = sorted_history['time'].diff()
gap_threshold = pd.Timedelta(seconds=1)
sorted_history['has_time_gap'] = (sorted_history['time_diff'] > gap_threshold).fillna(False).astype(int)
sorted_history.head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,...,geoaltitude,lastposupdate,lastcontact,serials,hour,year,month,day,time_diff,has_time_gap
21102,2024-08-08 13:55:00+00:00,aa73a0,30.593079,-93.695715,240.144531,236.888658,-11.70432,SWA16,False,False,...,11826.24,1723125285.654,1723125285.768,[-1408230187],13,2024,8,8,<NA>,0
21103,2024-08-08 13:55:01+00:00,aa73a0,30.593079,-93.695715,240.144531,236.888658,-11.70432,SWA16,False,False,...,11826.24,1723125285.654,1723125285.768,[],13,2024,8,8,0 days 00:00:01,0
21104,2024-08-08 13:55:02+00:00,aa73a0,30.593079,-93.695715,240.144531,236.888658,-11.70432,SWA16,False,False,...,11826.24,1723125285.654,1723125285.768,[],13,2024,8,8,0 days 00:00:01,0
21105,2024-08-08 13:55:03+00:00,aa73a0,30.593079,-93.695715,240.144531,236.888658,-11.70432,SWA16,False,False,...,11826.24,1723125285.654,1723125285.768,[],13,2024,8,8,0 days 00:00:01,0
21106,2024-08-08 13:55:04+00:00,aa73a0,30.593079,-93.695715,240.144531,236.888658,-11.70432,SWA16,False,False,...,11826.24,1723125285.654,1723125285.768,[],13,2024,8,8,0 days 00:00:01,0


In [24]:
sorted_history[sorted_history['has_time_gap'] == 1].head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,...,geoaltitude,lastposupdate,lastcontact,serials,hour,year,month,day,time_diff,has_time_gap
9301,2024-08-08 14:04:31+00:00,aa73a0,35.047818,-106.614761,187.941116,252.805026,-10.72896,SWA1652,False,False,...,1623.06,1723143572.591,1723125870.613,[-1408230850],14,2024,8,8,0 days 00:04:32,1
13862,2024-08-08 15:33:52+00:00,aa73a0,29.683548,-95.324038,120.450227,325.08817,4.22656,SWA16,False,False,...,312.42,1723126721.195,1723131231.045,[-1408235118],15,2024,8,8,0 days 01:10:11,1
0,2024-08-08 16:00:02+00:00,aa73a0,32.306741,-96.210919,174.981304,314.404431,-10.07872,SWA16,False,False,...,3726.18,1723132801.654,1723132801.731,"[-1408236845, -1408237160]",16,2024,8,8,0 days 00:00:03,1
29027,2024-08-08 17:13:21+00:00,aa73a0,32.830267,-96.839101,88.432011,136.650039,14.30528,SWA236,False,False,...,-7.62,1723137200.862,1723137200.862,[-1408236845],17,2024,8,8,0 days 00:54:13,1
6666,2024-08-08 18:00:01+00:00,aa73a0,34.522385,-103.062856,238.123119,278.322663,0.0,SWA16,False,False,...,12291.06,1723139998.967,1723140000.847,[-1408236882],18,2024,8,8,0 days 00:00:02,1


In [25]:
# Split records by time gaps
sorted_history['flight_leg'] = sorted_history['has_time_gap'].cumsum()
flight_legs = sorted_history.groupby('flight_leg')
for leg_id, leg_data in flight_legs:
    print(f'Leg {leg_id}: {len(leg_data):,} track points')

Leg 0: 300 track points
Leg 1: 1,151 track points
Leg 2: 1,568 track points
Leg 3: 1,147 track points
Leg 4: 2,799 track points
Leg 5: 2,308 track points
Leg 6: 327 track points
Leg 7: 660 track points
Leg 8: 1,716 track points
Leg 9: 3,598 track points
Leg 10: 507 track points
Leg 11: 3,352 track points
Leg 12: 58 track points
Leg 13: 1,153 track points
Leg 14: 1,149 track points
Leg 15: 151 track points
Leg 16: 3,572 track points
Leg 17: 2,100 track points
Leg 18: 905 track points
Leg 19: 3,305 track points


#### Plot Track Points on Map

In [26]:
# Create a base map centered at the midpoint of the flight track
map_center = [history_results['lat'].mean(), history_results['lon'].mean()]
m = folium.Map(
    location=map_center, 
    zoom_start=12,
    tiles="CartoDB dark_matter",
    height="60%"
)

# Plot each point with small circle markers and dynamic popup text
# for _, row in history_results.sort_values('time', ascending=True).iterrows():
#     # Create dynamic popup text based on the dataframe's columns
#     popup_text = '<br>'.join([f"{col}: {row[col]}" for col in history_results.columns])
    
#     # Add a small circle marker with a popup
#     folium.CircleMarker(
#         location=[row['lat'], row['lon']],
#         radius=3,
#         color='blue',
#         fill=True,
#         fill_color='blue',
#         fill_opacity=0.7,
#         popup=Popup(popup_text, max_width=300)
#     ).add_to(m)

# Add green polyline for the flight track
# flight_path = history_results.sort_values('time', ascending=True)[['lat', 'lon']].values.tolist()
# folium.PolyLine(flight_path, color="green", weight=2.5, opacity=1).add_to(m)

# Add green polying for each flight leg
for leg_id, leg_data in flight_legs:
    flight_path = leg_data.sort_values('time', ascending=True)[['lat', 'lon']].values.tolist()
    folium.PolyLine(flight_path, color="green", weight=2.5, opacity=1).add_to(m)

m

### Getting Flight Track Points Data in Bulk
The goal is to be able to iteratively save track points data from 2017 through 2024. It's expected that the dataset is too large to fit in a single dataframe. Therefore, iterating from 01/01/2017 to 10/05/2024 in 1 hour increments, saving each data extract to a file with a folder structure partitioning by year, month, day, hour

In [49]:
# Determine number of years between dates
(datetime.datetime.strptime('2024-01-01 10:00:00', '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')).days / 365

5.002739726027397

In [45]:
output_path = 's3://endurasoft-dev-risk-framework/opensky-network/track-points'
flights_per_request = 200 # 500, 200, 100
hours_interval = 5, # 10, 5, 1

start_datetime = datetime.datetime.strptime('2017-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
end_datetime = datetime.datetime.strptime('2017-01-01 5:00:00', '%Y-%m-%d %H:%M:%S')

current_datetime = end_datetime # start_datetime
extract_num = 0
date_diff = end_datetime - start_datetime
days, seconds = date_diff.days, date_diff.seconds
hours = days * 24 + seconds // 3600
num_iters = hours // hours_interval
if hours % hours_interval != 0:
    num_iters += 1

with tqdm(total=num_iters) as pbar:
    # Work backwards so that latest data is prioritized first
    while current_datetime > start_datetime:
        pbar.update(1)
        start = current_datetime - datetime.timedelta(hours=hours_interval)

        if start < start_datetime:
            # print(f'{start} is less than {start_datetime}')
            start = start_datetime

        stop = current_datetime
        pbar.set_description(f'Obtaining Extraction {extract_num+1} of {num_iters}: {start} to {stop}', refresh=True)
        # print(f'Obtaining Extraction {extract_num+1} of {num_iters}: {start} to {stop}')

        # Make request for flight profiles
        flight_profiles = trino.flightlist(
            start=start,
            stop=stop
        )
        
        # Filter flight profiles where departure and arrival are known and belong in the airports list
        filtered_profiles = flight_profiles[(flight_profiles['departure'].isin(airports_set)) & (flight_profiles['arrival'].isin(airports_set))]

        # Filter by random sample based on flights per request. If total number of track points is less than flights per request, do not randomly sample
        if len(filtered_profiles) >= flights_per_request:
            filtered_profiles = filtered_profiles.sample(flights_per_request)

        # Make request for flight track points for each unique flight id in the filtered flight profiles results and obtain results
        # NOTE: By default, date_delta splits requests by hour
        track_points = trino.history(
            start=start,
            stop=stop,
            icao24=list(set(filtered_profiles['icao24'].unique().tolist()))
        )

        # Create partition columns for year, month, day, hour
        track_points['year'] = track_points['time'].dt.year
        track_points['month'] = track_points['time'].dt.month
        track_points['day'] = track_points['time'].dt.day
        track_points['hour'] = track_points['time'].dt.hour

        # Save to s3 with specific partition columns
        pbar.set_description(f'Saving Extraction {extract_num+1} of {num_iters}: {start} to {stop}', refresh=True)
        wr.s3.to_parquet(
            df=track_points,
            path=output_path,
            dataset=True,
            partition_cols=["year", "month", "day", "hour"],
            compression="snappy"  # Optional compression
        )

        extract_num += 1
        current_datetime = start

FINISHED: : 100% [00:03, 26.8%/s]-01-01 00:00:00 to 2017-01-01 10:00:00: 100%|██████████| 1/1 [00:00<00:00, 22671.91it/s]
DOWNLOAD: 7.65klines [00:00, 279klines/s]
RUNNING: : 69.0% [00:17, 3.92%/s]
DOWNLOAD: 3.64Mlines [19:52, 3.05klines/s]
Saving Extraction 1 of 10: 2017-01-01 00:00:00 to 2017-01-01 10:00:00: 100%|██████████| 1/1 [20:46<00:00, 1246.97s/it]   


### Flight Raw Data (Messages)

In [13]:
start = start_datetime
flight_id = 'aa73a0'

rawdata_results = trino.rawdata(
    start=start,
    icao24=flight_id
)

print(f'Raw data records from {start} for flight id "{flight_id}": {len(rawdata_results):,}')

No credentials provided, falling back to browser authentication


Open the following URL in browser for the external authentication:
https://trino.opensky-network.org/oauth2/token/initiate/e50dd22dd2f5d8b8889e03b3e192c7c94988cabdff387a4c2631e16943eab83e


FINISHED: : 100% [00:00, 106%/s] 
DOWNLOAD: 31.0lines [00:00, 5.40klines/s]

Raw data records from 2024-08-08 13:55:00 for flight id "aa73a0": 31


In [14]:
rawdata_results.head()


,mintime,rawmsg,icao24
0,1723175070.767,200001b9ddbbb2,aa73a0
1,1723175131.305,2000013c25714d,aa73a0
2,1723175112.304,20000195dc3258,aa73a0
3,1723175112.924,20000195dc3258,aa73a0
4,1723175072.492,200001b8224fbb,aa73a0
